In [1]:
import pandas as pd
import sqlite3
import json, csv
#import geopandas
import re
import numpy as np
import time

In [2]:
address_points = pd.read_csv('cook_locations.csv')
address_points.head()

/tmp/ipykernel_71927/3716568878.py:1: DtypeWarning: Columns (1,5,7,11,17,25,26,27,30,31,32,36,38,40,46,47,48,52,54) have mixed types. Specify dtype option on import or set low_memory=False.
  address_points = pd.read_csv('cook_locations.csv')


,OBJECTID,AddNum_Pre,Add_Number,AddNum_Suf,ADDRNOCOM,St_PreMod,St_PreDir,St_PreTyp,St_Name,St_PosTyp,...,Seat,Inc_Muni,Uninc_Comm,MSAGComm,DiscrpAgID,Site_NGUID,ESN,TWP_NAME,LSt_PreDir,LSt_Type
0,1,NaN,1763.0,NaN,1763,NaN,WEST,NaN,WELLINGTON,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1622.0,NaN,1622,NaN,WEST,NaN,CULLERTON,STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,917.0,NaN,917,NaN,WEST,NaN,ARMITAGE,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,20018.0,NaN,20018,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,20024.0,NaN,20024,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
address_points.columns

Index(['OBJECTID', 'AddNum_Pre', 'Add_Number', 'AddNum_Suf', 'ADDRNOCOM',
       'St_PreMod', 'St_PreDir', 'St_PreTyp', 'St_Name', 'St_PosTyp',
       'St_PosDir', 'St_PosMod', 'STNAMECOM', 'LandmkName', 'PLACENAME',
       'Post_Comm', 'State', 'Post_Code', 'Post_Code4', 'ADDRDELIV',
       'CMPADDABRV', 'Long', 'Lat', 'USGRIDCORD', 'Elev', 'PIN', 'Effective',
       'Expire', 'EDITTIME', 'geocode_muni', 'Document_Name', 'created_user',
       'created_date', 'last_edited_user', 'last_edited_date', 'Source',
       'Source_Comment', 'Structure', 'OrthoYear', 'Placement', 'County',
       'Building', 'Floor', 'Unit', 'Room', 'Seat', 'Inc_Muni', 'Uninc_Comm',
       'MSAGComm', 'DiscrpAgID', 'Site_NGUID', 'ESN', 'TWP_NAME', 'LSt_PreDir',
       'LSt_Type'],
      dtype='object')

In [4]:
cook_locations = address_points[['Add_Number', 'ADDRDELIV','CMPADDABRV','Lat', 'Long','PLACENAME','Post_Code']]
cook_locations.head()

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461


In [5]:
cook_locations.dtypes

Add_Number    float64
ADDRDELIV      object
CMPADDABRV     object
Lat           float64
Long          float64
PLACENAME      object
Post_Code      object
dtype: object

In [6]:
cook_locations.shape

(1347092, 7)

In [7]:
#get rid of nan values

In [7]:
cook_locations_clean = cook_locations[~cook_locations['Post_Code'].isnull()]
cook_locations_clean

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461
...,...,...,...,...,...,...,...
1347087,222.0,222 MERCHANDISE MART PLAZA,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654
1347088,1410.0,1410 SOUTH MUSEUM CAMPUS DRIVE,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605
1347089,NaN,6660 WEST 66TH STREET,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638
1347090,9590.0,9590 WEST 55TH STREET,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525


In [9]:
#get rid of empty strings
#cast as a string
#check the success rate if it about 90%


In [10]:
#cook_locations_clean[cook_locations_clean['Post_Code'] == ' ']

In [15]:
#cook_locations_clean = cook_locations_clean[cook_locations_clean['Post_Code'] != ' ']
#cook_locations_clean

,Add_Number,ADDRDELIV,CMPADDABRV,Lat,Long,PLACENAME,Post_Code
0,1763.0,1763 WEST WELLINGTON AVENUE,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657
1,1622.0,1622 WEST CULLERTON STREET,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608
2,917.0,917 WEST ARMITAGE AVENUE,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614
3,20018.0,20018 MOHAWK TRAIL,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461
4,20024.0,20024 MOHAWK TRAIL,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461
...,...,...,...,...,...,...,...
1347087,222.0,222 MERCHANDISE MART PLAZA,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654
1347088,1410.0,1410 SOUTH MUSEUM CAMPUS DRIVE,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605
1347089,NaN,6660 WEST 66TH STREET,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638
1347090,9590.0,9590 WEST 55TH STREET,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525


In [8]:
cook_locations_clean.dtypes

Add_Number    float64
ADDRDELIV      object
CMPADDABRV     object
Lat           float64
Long          float64
PLACENAME      object
Post_Code      object
dtype: object

In [18]:
#cook_locations_clean['Post_Code'] = cook_locations_clean['Post_Code'].astype(str).astype(int)

/tmp/ipykernel_441706/3008046163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cook_locations_clean['Post_Code'] = cook_locations_clean['Post_Code'].astype(str).astype(int)


In [9]:
omec_locations = pd.read_csv('oemc_locations.csv')
omec_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD


In [10]:
omec_locations.shape

(171266, 12)

In [11]:
omec_locations.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical           object
directional         object
street_name         object
suffix              object
dtype: object

In [12]:
#check for alphabets in numerical column

In [13]:
omec_locations[omec_locations['numerical'].str.isalpha()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
1250,2000801443,01/08/2020 03.15.54,THREAS,THREATENING SUICIDE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
33053,2017807536,06/26/2020 13.27.24,ASLTIP,ASSAULT IP,FILE,RIDGE & AUSTIN,NaN,NaN,RIDGE,&,NaN,AUSTIN
34323,2018417148,07/02/2020 22.13.10,ATSUCD,ATTEMPT SUICIDE,7I,TRUMP INTERNATIONAL HOTE,1834 1832,A42,TRUMP,INTERNATIONAL,NaN,HOTE
40557,2021610499,08/03/2020 17.14.11,NPS,NO POLICE SERVICE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
67246,2035401094,12/19/2020 01.30.42,THREAS,THREATENING SUICIDE,FILE,CLAYTON MOTEL/ SNOW AVENUE,LOVES PARK,IL,CLAYTON,MOTEL/,SNOW,AVENUE
77674,2104513267,02/14/2021 22.17.20,THREAS,THREATENING SUICIDE,FILE,NEW ENGLAND/ X TH,NaN,NaN,NEW,ENGLAND/,X,TH
98438,2114909460,05/29/2021 14.13.31,DISTME,MENTAL HEALTH DISTURBANCE,FILE,W 87TH ST,NaN,NaN,W,87TH,NaN,ST
108404,2119503444,07/14/2021 08.44.18,DISTME,MENTAL HEALTH DISTURBANCE,19P,ILLINOIS MCCLURG CT,1802S 1823 1834,NaN,ILLINOIS,MCCLURG,NaN,CT
109196,2119812628,07/17/2021 18.51.00,MISSPE,MISSING PERSON,NaN,TRAINING EXERCISE ONLY,NaN,NaN,TRAINING,EXERCISE,NaN,ONLY
113218,2121707009,08/05/2021 12.48.58,DEATHR,DEATH REMOVAL,NaN,SWEDISH COVENANT HOSPITAL,1701,NaN,SWEDISH,COVENANT,NaN,HOSPITAL


In [14]:
omec_locations[omec_locations['numerical'].str.isupper()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
1250,2000801443,01/08/2020 03.15.54,THREAS,THREATENING SUICIDE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
33053,2017807536,06/26/2020 13.27.24,ASLTIP,ASSAULT IP,FILE,RIDGE & AUSTIN,NaN,NaN,RIDGE,&,NaN,AUSTIN
34323,2018417148,07/02/2020 22.13.10,ATSUCD,ATTEMPT SUICIDE,7I,TRUMP INTERNATIONAL HOTE,1834 1832,A42,TRUMP,INTERNATIONAL,NaN,HOTE
36638,2019615318,07/14/2020 21.29.09,SUSAUW,SUSPICIOUS AUTO WITH OCC,19PZ,N1198 N CICERO AV /48XX W DIVISION ST,1533,NaN,N1198,N,CICERO AV /48XX W DIVISION,ST
40557,2021610499,08/03/2020 17.14.11,NPS,NO POLICE SERVICE,FILE,JESSE BROWN VA HOSPITAL,NaN,NaN,JESSE,BROWN,VA,HOSPITAL
41322,2022004972,08/07/2020 10.44.10,CHILDP,CHILD PLAYING ON PHONE,FILE,A655 W 65TH ST,NaN,NaN,A655,W,65TH,ST
67246,2035401094,12/19/2020 01.30.42,THREAS,THREATENING SUICIDE,FILE,CLAYTON MOTEL/ SNOW AVENUE,LOVES PARK,IL,CLAYTON,MOTEL/,SNOW,AVENUE
77674,2104513267,02/14/2021 22.17.20,THREAS,THREATENING SUICIDE,FILE,NEW ENGLAND/ X TH,NaN,NaN,NEW,ENGLAND/,X,TH
97041,2114221111,05/22/2021 23.41.27,THREAS,THREATENING SUICIDE,FILE,AS5700-57XX W SCHOOL ST,NaN,NaN,AS5700-5700,W,SCHOOL,ST
98438,2114909460,05/29/2021 14.13.31,DISTME,MENTAL HEALTH DISTURBANCE,FILE,W 87TH ST,NaN,NaN,W,87TH,NaN,ST


In [15]:
omec_clean_locations = omec_locations[~omec_locations['numerical'].str.isupper()]

In [16]:
omec_clean_locations.shape

(171248, 12)

In [17]:
omec_clean_locations[~omec_clean_locations['numerical'].str.isnumeric()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
21419,2011910398,04/28/2020 17.56.09,DIST,DISTURBANCE,5FZ,\2844 N BROADWAY,1935,NaN,\2844,N,NaN,BROADWAY


21419    \2844 N BROADWAY
Name: Location, dtype: object

In [ ]:
#.str.strip
#check for

In [18]:
omec_clean_locations = omec_clean_locations[omec_clean_locations['EventNumber'] != 2011910398 ]

In [19]:
omec_clean_locations.shape

(171247, 12)

In [20]:
omec_clean_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD


In [21]:
omec_clean_locations['clean_street_name'] = omec_clean_locations['street_name']

In [22]:
omec_clean_locations.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,IRVING PARK


In [23]:
#omec_clean_locations['numerical'] = omec_clean_locations['numerical'].astype(str).astype(int)

In [23]:
omec_clean_locations['numerical'] = omec_clean_locations['numerical'].astype(str)

In [24]:
omec_clean_locations['numerical'] = omec_clean_locations['numerical'].str.strip()

In [25]:
omec_clean_locations['numerical'] = omec_clean_locations['numerical'].astype(int)

In [26]:
omec_clean_locations.dtypes

EventNumber           int64
EntryDate            object
EventType            object
TypeDescription      object
FinalDisposition     object
Location             object
CPDUnitList          object
CFDUnitList          object
numerical             int64
directional          object
street_name          object
suffix               object
clean_street_name    object
dtype: object

In [27]:
omec_clean_locations.shape

(171247, 13)

In [30]:
omec_nonS = omec_clean_locations[~omec_clean_locations['street_name'].str.contains("/",na=False)]
omec_nonS.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,IRVING PARK


In [31]:
omec_s = omec_clean_locations[omec_clean_locations['street_name'].str.contains("/",na=False)]
omec_s.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
16,2000102204,01/01/2020 01.59.07,THREAS,THREATENING SUICIDE,19P,5XX N CLARK ST /1XX W GRAND AV,1860,NaN,500,N,CLARK ST /1XX W GRAND,AV,CLARK ST /1XX W GRAND
28,2000104170,01/01/2020 03.47.33,THREAS,THREATENING SUICIDE,19B,7XX E 87TH ST /87XX S COTTAGE GROVE AV,631R,NaN,700,E,87TH ST /87XX S COTTAGE GROVE,AV,87TH ST /87XX S COTTAGE GROVE
37,2000105293,01/01/2020 05.19.12,BATIP,BATTERY IP,19IZ,4XX E 63RD ST /62XX S VERNON AV,320R 312R 313R,NaN,400,E,63RD ST /62XX S VERNON,AV,63RD ST /62XX S VERNON
56,2000107341,01/01/2020 09.30.08,MISSPE,MISSING PERSON,19P,36XX N SAYRE AV /69XX W ADDISON ST,1624 1630 1634 1683 1632 1652,NaN,3600,N,SAYRE AV /69XX W ADDISON,ST,SAYRE AV /69XX W ADDISON
74,2000109657,01/01/2020 12.54.32,CHECWB,CHECK WELL BEING,5079Z,16XX N DRAKE AV /35XX W NORTH AV,1423 1405 1411 1414 2535 1481D X1423 1412 1431...,NaN,1600,N,DRAKE AV /35XX W NORTH,AV,DRAKE AV /35XX W NORTH


In [32]:
omec_s.shape

(17874, 13)

In [33]:
omec_nonS.shape

(153373, 13)

In [ ]:
#clean up street names with slashes but removing everything after the slash in location

In [35]:
omec_s['clean_street_name'] = omec_s['street_name'].str.replace("(?:/).*",'',regex=True)

/tmp/ipykernel_71927/443209225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omec_s['clean_street_name'] = omec_s['street_name'].str.replace("(?:/).*",'',regex=True)


In [ ]:
171247

In [34]:
153373 + 17874

171247

In [36]:
omec_s.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
16,2000102204,01/01/2020 01.59.07,THREAS,THREATENING SUICIDE,19P,5XX N CLARK ST /1XX W GRAND AV,1860,NaN,500,N,CLARK ST /1XX W GRAND,AV,CLARK ST
28,2000104170,01/01/2020 03.47.33,THREAS,THREATENING SUICIDE,19B,7XX E 87TH ST /87XX S COTTAGE GROVE AV,631R,NaN,700,E,87TH ST /87XX S COTTAGE GROVE,AV,87TH ST
37,2000105293,01/01/2020 05.19.12,BATIP,BATTERY IP,19IZ,4XX E 63RD ST /62XX S VERNON AV,320R 312R 313R,NaN,400,E,63RD ST /62XX S VERNON,AV,63RD ST
56,2000107341,01/01/2020 09.30.08,MISSPE,MISSING PERSON,19P,36XX N SAYRE AV /69XX W ADDISON ST,1624 1630 1634 1683 1632 1652,NaN,3600,N,SAYRE AV /69XX W ADDISON,ST,SAYRE AV
74,2000109657,01/01/2020 12.54.32,CHECWB,CHECK WELL BEING,5079Z,16XX N DRAKE AV /35XX W NORTH AV,1423 1405 1411 1414 2535 1481D X1423 1412 1431...,NaN,1600,N,DRAKE AV /35XX W NORTH,AV,DRAKE AV


In [37]:
#create function to split upt clean_street_name and then just grab the street name
def splitUp(word):
    w = word.split(' ')
    return w[0]
    

In [32]:
#location_clean.split()

In [39]:
omec_s['clean_street_name'] = omec_s['clean_street_name'].apply(splitUp)

/tmp/ipykernel_71927/2911244237.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omec_s['clean_street_name'] = omec_s['clean_street_name'].apply(splitUp)


In [40]:
omec_s.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
16,2000102204,01/01/2020 01.59.07,THREAS,THREATENING SUICIDE,19P,5XX N CLARK ST /1XX W GRAND AV,1860,NaN,500,N,CLARK ST /1XX W GRAND,AV,CLARK
28,2000104170,01/01/2020 03.47.33,THREAS,THREATENING SUICIDE,19B,7XX E 87TH ST /87XX S COTTAGE GROVE AV,631R,NaN,700,E,87TH ST /87XX S COTTAGE GROVE,AV,87TH
37,2000105293,01/01/2020 05.19.12,BATIP,BATTERY IP,19IZ,4XX E 63RD ST /62XX S VERNON AV,320R 312R 313R,NaN,400,E,63RD ST /62XX S VERNON,AV,63RD
56,2000107341,01/01/2020 09.30.08,MISSPE,MISSING PERSON,19P,36XX N SAYRE AV /69XX W ADDISON ST,1624 1630 1634 1683 1632 1652,NaN,3600,N,SAYRE AV /69XX W ADDISON,ST,SAYRE
74,2000109657,01/01/2020 12.54.32,CHECWB,CHECK WELL BEING,5079Z,16XX N DRAKE AV /35XX W NORTH AV,1423 1405 1411 1414 2535 1481D X1423 1412 1431...,NaN,1600,N,DRAKE AV /35XX W NORTH,AV,DRAKE


In [43]:
omec_complete = pd.concat([omec_nonS,omec_s])
omec_complete

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,IRVING PARK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171204,2215812221,06/07/2022 17.47.42,DISTME,MENTAL HEALTH DISTURBANCE,19E,40XX N WESTERN AV / 24XX W CUYLER AV,1921,NaN,4000,N,WESTERN AV / 24XX W CUYLER,AV,WESTERN
171208,2215812568,06/07/2022 18.09.58,DISTME,MENTAL HEALTH DISTURBANCE,5B,23XX W DIVERSEY AV / 28XX N OAKLEY AV,1433 1421,NaN,2300,W,DIVERSEY AV / 28XX N OAKLEY,AV,DIVERSEY
171231,2215814431,06/07/2022 19.58.44,PERGUN,PERSON WITH A GUN,19PZ,23XX W 95TH ST / 94XX S WESTERN AV,4264A 2262A 2262D,NaN,2300,W,95TH ST / 94XX S WESTERN,AV,95TH
171245,2215816273,06/07/2022 21.38.45,SHOTSF,SHOTS FIRED,19PZ,58XX N BROADWAY / 11XX W ARDMORE AV,2020,NaN,5800,N,BROADWAY / 11XX W ARDMORE,AV,BROADWAY


In [42]:
omec_clean_locations.shape

(171247, 13)

In [35]:
omec_clean_locations.shape

(171247, 13)

In [ ]:
#change the address of Martin Luther king drive to DOCTOR MARTIN LUTHER KING DRIVE

In [36]:
#omec_clean_locations['clean_street_name'] = omec_clean_locations['clean_street_name'].str.replace('DR MARTIN LUTHER KING JR', 'DOCTOR MARTIN LUTHER KING JR')

In [44]:
#strip the any trailing white space
omec_complete['directional'] = omec_complete['directional'].str.strip()

In [45]:
omec_complete.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,IRVING PARK


In [46]:
def fullDirections(direction):
    if direction == 'W':
        return 'WEST'
    if direction == 'E':
        return 'EAST'
    if direction == 'S':
        return 'SOUTH'
    if direction == 'N':
        return 'NORTH'
    else:
        return direction
    

In [47]:
omec_complete['directional']= omec_complete['directional'].apply(fullDirections)
omec_complete.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,SOUTH,HALSTED,ST,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,SOUTH,SPAULDING,AV,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,WEST,SCHOOL,ST,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,WEST,IRVING PARK,RD,IRVING PARK


In [48]:
omec_complete.tail()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
171204,2215812221,06/07/2022 17.47.42,DISTME,MENTAL HEALTH DISTURBANCE,19E,40XX N WESTERN AV / 24XX W CUYLER AV,1921,NaN,4000,NORTH,WESTERN AV / 24XX W CUYLER,AV,WESTERN
171208,2215812568,06/07/2022 18.09.58,DISTME,MENTAL HEALTH DISTURBANCE,5B,23XX W DIVERSEY AV / 28XX N OAKLEY AV,1433 1421,NaN,2300,WEST,DIVERSEY AV / 28XX N OAKLEY,AV,DIVERSEY
171231,2215814431,06/07/2022 19.58.44,PERGUN,PERSON WITH A GUN,19PZ,23XX W 95TH ST / 94XX S WESTERN AV,4264A 2262A 2262D,NaN,2300,WEST,95TH ST / 94XX S WESTERN,AV,95TH
171245,2215816273,06/07/2022 21.38.45,SHOTSF,SHOTS FIRED,19PZ,58XX N BROADWAY / 11XX W ARDMORE AV,2020,NaN,5800,NORTH,BROADWAY / 11XX W ARDMORE,AV,BROADWAY
171257,2215817099,06/07/2022 22.30.09,SHOTSF,SHOTS FIRED,19PZ,23XX W 79TH ST / 78XX S WESTERN AV,863C,NaN,2300,WEST,79TH ST / 78XX S WESTERN,AV,79TH


In [27]:
#omec_clean_locations[omec_clean_locations['street_name'] == 'DR MARTIN LUTHER KING JR']['suffix'] = ''

/tmp/ipykernel_51235/1182088338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  omec_clean_locations[omec_clean_locations['street_name'] == 'DR MARTIN LUTHER KING JR']['suffix'] = ''


In [49]:
omec_complete.to_csv('cleaned_oemc_locations2.csv')

In [29]:
#check out cook county address data and subset to columns needed for geocoding

In [50]:
address_points.head()

,OBJECTID,AddNum_Pre,Add_Number,AddNum_Suf,ADDRNOCOM,St_PreMod,St_PreDir,St_PreTyp,St_Name,St_PosTyp,...,Seat,Inc_Muni,Uninc_Comm,MSAGComm,DiscrpAgID,Site_NGUID,ESN,TWP_NAME,LSt_PreDir,LSt_Type
0,1,NaN,1763.0,NaN,1763,NaN,WEST,NaN,WELLINGTON,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1622.0,NaN,1622,NaN,WEST,NaN,CULLERTON,STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,917.0,NaN,917,NaN,WEST,NaN,ARMITAGE,AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,20018.0,NaN,20018,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,20024.0,NaN,20024,NaN,NaN,NaN,MOHAWK,TRAIL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
address_points.columns

Index(['OBJECTID', 'AddNum_Pre', 'Add_Number', 'AddNum_Suf', 'ADDRNOCOM',
       'St_PreMod', 'St_PreDir', 'St_PreTyp', 'St_Name', 'St_PosTyp',
       'St_PosDir', 'St_PosMod', 'STNAMECOM', 'LandmkName', 'PLACENAME',
       'Post_Comm', 'State', 'Post_Code', 'Post_Code4', 'ADDRDELIV',
       'CMPADDABRV', 'Long', 'Lat', 'USGRIDCORD', 'Elev', 'PIN', 'Effective',
       'Expire', 'EDITTIME', 'geocode_muni', 'Document_Name', 'created_user',
       'created_date', 'last_edited_user', 'last_edited_date', 'Source',
       'Source_Comment', 'Structure', 'OrthoYear', 'Placement', 'County',
       'Building', 'Floor', 'Unit', 'Room', 'Seat', 'Inc_Muni', 'Uninc_Comm',
       'MSAGComm', 'DiscrpAgID', 'Site_NGUID', 'ESN', 'TWP_NAME', 'LSt_PreDir',
       'LSt_Type'],
      dtype='object')

In [52]:
columns = ['CMPADDABRV', 'Lat', 'Long', 'PLACENAME', 'Post_Code', 'OBJECTID', 'Add_Number','St_PreDir', 'St_Name']

In [53]:
database = address_points[columns]
database 

,CMPADDABRV,Lat,Long,PLACENAME,Post_Code,OBJECTID,Add_Number,St_PreDir,St_Name
0,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657,1,1763.0,WEST,WELLINGTON
1,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608,2,1622.0,WEST,CULLERTON
2,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614,3,917.0,WEST,ARMITAGE
3,20018 MOHAWK TRL,41.525564,-87.695313,Olympia Fields,60461,4,20018.0,NaN,MOHAWK
4,20024 MOHAWK TRL,41.525317,-87.695269,Olympia Fields,60461,5,20024.0,NaN,MOHAWK
...,...,...,...,...,...,...,...,...,...
1347087,222 MERCHANDISE MART PLZ,NaN,NaN,CHICAGO,60654,1465915,222.0,NaN,MERCHANDISE MART
1347088,1410 S MUSEUM CAMPUS DR,NaN,NaN,CHICAGO,60605,1465916,1410.0,SOUTH,MUSEUM CAMPUS
1347089,6660 W 66TH ST,NaN,NaN,BEDFORD PARK,60638,1465917,NaN,WEST,66TH
1347090,9590 W 55TH ST,NaN,NaN,COUNTRYSIDE,60525,1465918,9590.0,WEST,55TH


In [54]:
database = database[database['PLACENAME'] =='Chicago']

In [55]:
database.shape

(582251, 9)

## Chunking the dataframes into mini Dataframes

In [48]:
#split array into chunks in order to run the process on my laptop

In [74]:
chunkDf = np.array_split(omec_complete,30)
chunkDf[0]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,SOUTH,HALSTED,ST,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,SOUTH,SPAULDING,AV,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,WEST,SCHOOL,ST,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,WEST,IRVING PARK,RD,IRVING PARK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6255,2003611162,02/05/2020 16.32.27,THREAS,THREATENING SUICIDE,5079Z,119XX S EGGLESTON AV,510 571,A5,11900,SOUTH,EGGLESTON,AV,EGGLESTON
6256,2003611211,02/05/2020 16.35.31,DIST,DISTURBANCE,5FZ,40XX N LINCOLN AV,1911,NaN,4000,NORTH,LINCOLN,AV,LINCOLN
6257,2003611262,02/05/2020 16.39.13,DISTME,MENTAL HEALTH DISTURBANCE,19E,22XX W BYRON ST,1922 1912,NaN,2200,WEST,BYRON,ST,BYRON
6258,2003611471,02/05/2020 16.51.26,BATJO,BATTERY JO,19PZ,66XX S SACRAMENTO AV,872,NaN,6600,SOUTH,SACRAMENTO,AV,SACRAMENTO


In [56]:
#chunkDf = np.array_split(omec_complete,3000)
#chunkDf[0]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,SOUTH,HALSTED,ST,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,SOUTH,SPAULDING,AV,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,WEST,SCHOOL,ST,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,WEST,IRVING PARK,RD,IRVING PARK
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,NORTH,SHERIDAN,RD,SHERIDAN
6,2000100941,01/01/2020 00.49.19,BATIP,BATTERY IP,1EZ,66XX W BELDEN AV,2512R 2510R,NaN,6600,WEST,BELDEN,AV,BELDEN
7,2000100959,01/01/2020 00.50.02,THREAS,THREATENING SUICIDE,16I,5XX N AUSTIN BL,1511R 1512R,NaN,500,NORTH,AUSTIN,BL,AUSTIN
8,2000100999,01/01/2020 00.52.12,DISTME,MENTAL HEALTH DISTURBANCE,5079Z,50XX S MICHIGAN AV,224R,NaN,5000,SOUTH,MICHIGAN,AV,MICHIGAN
9,2000101032,01/01/2020 00.53.27,DD,DOMESTIC DISTURBANCE,1IZ,10XX N DEARBORN ST,1824R 1823R,NaN,1000,NORTH,DEARBORN,ST,DEARBORN


## Helper Functions

In [ ]:
#create lambda function and use .apply instead of iterrow

In [75]:
chunkHead = chunkDf[0].head()
chunkHead

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,SOUTH,HALSTED,ST,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,SOUTH,SPAULDING,AV,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,WEST,SCHOOL,ST,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,WEST,IRVING PARK,RD,IRVING PARK


In [57]:
database.head()

,CMPADDABRV,Lat,Long,PLACENAME,Post_Code,OBJECTID,Add_Number,St_PreDir,St_Name
0,1763 W WELLINGTON AVE,41.935796,-87.673357,Chicago,60657,1,1763.0,WEST,WELLINGTON
1,1622 W CULLERTON ST,41.855258,-87.667139,Chicago,60608,2,1622.0,WEST,CULLERTON
2,917 W ARMITAGE AVE,41.917926,-87.651651,Chicago,60614,3,917.0,WEST,ARMITAGE
6,1132 S WABASH AVE,41.868253,-87.626334,Chicago,60605,7,1132.0,SOUTH,WABASH
7,732 N WILLARD CT,41.895400,-87.658712,Chicago,60642,8,732.0,NORTH,WILLARD


In [58]:
def geoCodeChunk(numerical,direction,street_name,database_df):
    
    complete_conditional = database_df[(database_df['St_Name'] == street_name) 
                                & (database_df['St_PreDir'] == direction)
                                & (database_df['Add_Number'] < numerical + 100)
                                & (database_df['Add_Number'] >= numerical)
                                ]
    
    '''
     Improve selection criteria from the result set
    '''
    
    
    list_zipcodes = list(set(complete_conditional['Post_Code']))
    list_lat = list(set(complete_conditional['Lat']))
    list_long = list(set(complete_conditional['Long']))
    
    
    return [list_zipcodes, list_lat,list_long]

In [59]:
def geoCodeToDf(chunkDfName,columnNamesList):
    #chunkDfName[newColumnName]= chunkZipCodeList
    newDataframe = pd.DataFrame.from_records(chunkDfName, columns=[columnNamesList])
    return newDataframe
    #chunkDfName[number].to_csv(f'geocoded_batches/batch{number}.csv')

In [60]:
def extractElement(columnList):
    try:
        return columnList[0]
    except:
        return None

In [61]:
def extractElement1(columnList):
    try:
        return columnList[0][0]
    except:
        return None

In [62]:
def extractElement2(columnList):
    try:
        return columnList[1][0]
    except:
        return None

In [63]:
def extractElement3(columnList):
    try:
        return columnList[2][0]
    except:
        return None

In [64]:
#creates 30 empty list 
#name convention list_zipcodes0, list_zipcodes1,..etc
for k in range(30):
    exec(f'list_results{k} = []')

In [65]:
oemc_output = None

In [76]:
oemc_output = pd.DataFrame(columns=['EventNumber', 'EntryDate', 'EventType', 'TypeDescription', 'FinalDisposition', 'Location', 'CPDUnitList', 'CFDUnitList', 'numerical', 'directional', 'street_name', 'suffix' , 'zipcodes','lats','longs','clean_street_name'])

In [77]:
oemc_output

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,clean_street_name


In [102]:
chunkDf[0].head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,clean_street_name,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,NaN,SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,NaN,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,NaN,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,NaN,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,NaN,WEST


In [68]:
list_results = chunkDf[0].apply(lambda x: geoCodeChunk(x["numerical"],x["directional"],x["clean_street_name"],database),axis=1)


In [69]:
list_results

0     [[60620], [41.75590425, 41.75534803, 41.754398...
1     [[60655], [41.70491317, 41.70578963, 41.704799...
2     [[60641], [41.94070996, 41.94066717, 41.940666...
3                                          [[], [], []]
4     [[60634], [41.95379298, 41.95332232, 41.953322...
5     [[60640], [41.97354658, 41.9740266, 41.9749648...
6              [[60707], [41.92089919], [-87.79111516]]
7     [[60644], [41.88970297, 41.89059431], [-87.774...
8     [[60615], [41.80248021, 41.80260776, 41.803058...
9     [[60610], [41.90146595, 41.9013319, 41.9014002...
10    [[60618], [41.93539619, 41.93436298, 41.934792...
11    [[60630], [41.96393208, 41.96433771, 41.964335...
12    [[60630], [41.97373059, 41.97329907, 41.973381...
13    [[60646], [41.99227562, 41.9921866, 41.9927537...
14    [[60629], [41.78304312, 41.78359008, 41.783249...
15    [[60652], [41.75302696, 41.75203789, 41.752276...
17    [[60621], [41.77791888, 41.77792686, 41.777926...
18    [[60615], [41.7958596, 41.79571297, 41.796

In [ ]:
error_index =[]

In [169]:
t = chunkDf[0].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)
t

0      [[60620], [41.75590425, 41.75534803, 41.754398...
1      [[60655], [41.70491317, 41.70578963, 41.704799...
2      [[60641], [41.94070996, 41.94066717, 41.940666...
3                                           [[], [], []]
4      [[60634], [41.95379298, 41.95332232, 41.953322...
                             ...                        
167    [[60649], [41.76369594, 41.76321687, 41.763708...
168    [[60643], [41.69454034, 41.69433534, 41.695667...
169    [[60707], [41.91339166, 41.91407821, 41.912790...
170    [[60628], [41.66393368, 41.66507118, 41.664654...
171    [[60617], [41.74480538, 41.74540141, 41.745195...
Length: 172, dtype: object

In [166]:
len(t[0])

3

In [173]:
t_df = t.to_frame('t')
t_df

,t
0,"[[60620], [41.75590425, 41.75534803, 41.754398..."
1,"[[60655], [41.70491317, 41.70578963, 41.704799..."
2,"[[60641], [41.94070996, 41.94066717, 41.940666..."
3,"[[], [], []]"
4,"[[60634], [41.95379298, 41.95332232, 41.953322..."
...,...
167,"[[60649], [41.76369594, 41.76321687, 41.763708..."
168,"[[60643], [41.69454034, 41.69433534, 41.695667..."
169,"[[60707], [41.91339166, 41.91407821, 41.912790..."
170,"[[60628], [41.66393368, 41.66507118, 41.664654..."


,t
0,"[[[60620], [41.75590425, 41.75534803, 41.75..."
1,"[[[60655], [41.70491317, 41.70578963, 41.70..."
2,"[[[60641], [41.94070996, 41.94066717, 41.94..."
3,"[[[], [], []]]"
4,"[[[60634], [41.95379298, 41.95332232, 41.95..."
...,...
167,"[[[60649], [41.76369594, 41.76321687, 41.76..."
168,"[[[60643], [41.69454034, 41.69433534, 41.69..."
169,"[[[60707], [41.91339166, 41.91407821, 41.91..."
170,"[[[60628], [41.66393368, 41.66507118, 41.66..."


In [171]:
t.iloc[3]

[[], [], []]

In [187]:
codes = t_df['t'].apply(extractElement1)

In [188]:
lats = t_df['t'].apply(extractElement2)

In [189]:
longs =t_df['t'].apply(extractElement3)

In [192]:
t_df['zipcodes'] = codes

In [193]:
t_df

,t,zipcodes
0,"[[60620], [41.75590425, 41.75534803, 41.754398...",[60620]
1,"[[60655], [41.70491317, 41.70578963, 41.704799...",[60655]
2,"[[60641], [41.94070996, 41.94066717, 41.940666...",[60641]
3,"[[], [], []]",[]
4,"[[60634], [41.95379298, 41.95332232, 41.953322...",[60634]
...,...,...
167,"[[60649], [41.76369594, 41.76321687, 41.763708...",[60649]
168,"[[60643], [41.69454034, 41.69433534, 41.695667...",[60643]
169,"[[60707], [41.91339166, 41.91407821, 41.912790...",[60707]
170,"[[60628], [41.66393368, 41.66507118, 41.664654...",[60628]


In [78]:
oemc_output.shape

(0, 16)

In [79]:
df = None

In [80]:
df_complete = None

In [81]:
for i in range(6):
    exec(f'df_results = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["directional"],x["clean_street_name"],database),axis=1)')
    df = df_results.to_frame(name='t')
    codes = df['t'].apply(extractElement1)
    df['zipcodes'] = codes
    
    lats = df['t'].apply(extractElement2)
    df['lats'] = lats
    
    longs = df['t'].apply(extractElement3)
    df['longs'] = longs
    
    df = df.drop('t', axis=1)
    

        #df_final = pd.DataFrame.from_records(df_results, columns=['zipcodes','lats','longs'])
    df_complete = pd.concat([chunkDf[i], df], axis=1)
        #df_complete['zipcode'] = df_complete['zipcodes'].apply(extractElement)
        #df_complete['lat'] = df_complete['lats'].apply(extractElement)
        #df_complete['long'] = df_complete['longs'].apply(extractElement)
    oemc_output = pd.concat([oemc_output,df_complete])

    print('Done')

Done
Done
Done
Done
Done
Done


In [82]:
oemc_output.shape

(34254, 16)

In [83]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,SOUTH,HALSTED,ST,60620,41.755904,-87.643854,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,SOUTH,SPAULDING,AV,60655,41.704913,-87.704104,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,WEST,SCHOOL,ST,60641,41.940710,-87.729486,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,None,NaN,NaN,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,WEST,IRVING PARK,RD,60634,41.953793,-87.790068,IRVING PARK


In [84]:
oemc_output.to_csv('oemc_output_data2.csv')

In [85]:
for i in range(6,12):
    exec(f'df_results = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["directional"],x["clean_street_name"],database),axis=1)')
    df = df_results.to_frame(name='t')
    codes = df['t'].apply(extractElement1)
    df['zipcodes'] = codes
    
    lats = df['t'].apply(extractElement2)
    df['lats'] = lats
    
    longs = df['t'].apply(extractElement3)
    df['longs'] = longs
    
    df = df.drop('t', axis=1)
    

        #df_final = pd.DataFrame.from_records(df_results, columns=['zipcodes','lats','longs'])
    df_complete = pd.concat([chunkDf[i], df], axis=1)
        #df_complete['zipcode'] = df_complete['zipcodes'].apply(extractElement)
        #df_complete['lat'] = df_complete['lats'].apply(extractElement)
        #df_complete['long'] = df_complete['longs'].apply(extractElement)
    oemc_output = pd.concat([oemc_output,df_complete])

    print('Done')

Done
Done
Done
Done
Done
Done


In [ ]:
oemc_output.head()

In [88]:
oemc_output.tail()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,clean_street_name
76637,2103911143,02/08/2021 18.44.42,DISTME,MENTAL HEALTH DISTURBANCE,5P,23XX W WASHINGTON BL,1268A,NaN,2300,WEST,WASHINGTON,BL,60612,41.882798,-87.684809,WASHINGTON
76638,2103911158,02/08/2021 18.46.04,DOMBAT,DOMESTIC BATTERY,19IZ,24XX S HOMAN AV,1031 1013,NaN,2400,SOUTH,HOMAN,AV,60623,41.847010,-87.709744,HOMAN
76639,2103911430,02/08/2021 19.10.00,DD,DOMESTIC DISTURBANCE,1365,16XX N TALMAN AV,1412 1424,NaN,1600,NORTH,TALMAN,AV,60647,41.911561,-87.693051,TALMAN
76640,2103911463,02/08/2021 19.13.18,PERKNI,PERSON WITH A KNIFE,7IZ,X E 102ND ST,510 511 524 571,NaN,0,EAST,102ND,ST,60628,41.708292,-87.621075,102ND
76641,2103911584,02/08/2021 19.22.38,DISTMU,DISTURBANCE - MUSIC/NOISE,4P,11XX N HARDING AV,1172,NaN,1100,NORTH,HARDING,AV,60651,41.900990,-87.725330,HARDING


In [86]:
oemc_output.shape

(68503, 16)

In [87]:
oemc_output.to_csv('oemc_output_data2.csv')

In [89]:
for i in range(12,18):
    exec(f'df_results = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["directional"],x["clean_street_name"],database),axis=1)')
    df = df_results.to_frame(name='t')
    codes = df['t'].apply(extractElement1)
    df['zipcodes'] = codes
    
    lats = df['t'].apply(extractElement2)
    df['lats'] = lats
    
    longs = df['t'].apply(extractElement3)
    df['longs'] = longs
    
    df = df.drop('t', axis=1)
    

    df_complete = pd.concat([chunkDf[i], df], axis=1)
    oemc_output = pd.concat([oemc_output,df_complete])

    print('Done')

Done
Done
Done
Done
Done
Done


In [90]:
oemc_output.shape

(102751, 16)

In [91]:
oemc_output.to_csv('oemc_output_data2.csv')

In [92]:
for i in range(18,24):
    exec(f'df_results = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["directional"],x["clean_street_name"],database),axis=1)')
    df = df_results.to_frame(name='t')
    codes = df['t'].apply(extractElement1)
    df['zipcodes'] = codes
    
    lats = df['t'].apply(extractElement2)
    df['lats'] = lats
    
    longs = df['t'].apply(extractElement3)
    df['longs'] = longs
    
    df = df.drop('t', axis=1)
    

        #df_final = pd.DataFrame.from_records(df_results, columns=['zipcodes','lats','longs'])
    df_complete = pd.concat([chunkDf[i], df], axis=1)
        #df_complete['zipcode'] = df_complete['zipcodes'].apply(extractElement)
        #df_complete['lat'] = df_complete['lats'].apply(extractElement)
        #df_complete['long'] = df_complete['longs'].apply(extractElement)
    oemc_output = pd.concat([oemc_output,df_complete])

    print('Done')

Done
Done
Done
Done
Done
Done


In [ ]:
#exec(f'df_results = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')


In [93]:
oemc_output.shape

(136999, 16)

In [94]:
oemc_output.to_csv('oemc_output_data2.csv')

In [70]:
28404/136999

0.20732998051080664

In [68]:
oemc_output.dtypes

EventNumber          object
EntryDate            object
EventType            object
TypeDescription      object
FinalDisposition     object
Location             object
CPDUnitList          object
CFDUnitList          object
numerical            object
directional          object
street_name          object
suffix               object
zipcodes             object
lats                float64
longs               float64
St_PreDir            object
dtype: object

In [69]:
oemc_output[oemc_output['zipcodes'].isnull()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,St_PreDir
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,None,NaN,NaN,.5
16,2000102204,01/01/2020 01.59.07,THREAS,THREATENING SUICIDE,19P,5XX N CLARK ST /1XX W GRAND AV,1860,NaN,500,N,CLARK ST /1XX W GRAND,AV,None,NaN,NaN,NORTH
26,2000103846,01/01/2020 03.28.07,DISTME,MENTAL HEALTH DISTURBANCE,15P,91XX S DR MARTIN LUTHER KING JR DR,632R,NaN,9100,S,DR MARTIN LUTHER KING JR,DR,None,NaN,NaN,SOUTH
28,2000104170,01/01/2020 03.47.33,THREAS,THREATENING SUICIDE,19B,7XX E 87TH ST /87XX S COTTAGE GROVE AV,631R,NaN,700,E,87TH ST /87XX S COTTAGE GROVE,AV,None,NaN,NaN,EAST
37,2000105293,01/01/2020 05.19.12,BATIP,BATTERY IP,19IZ,4XX E 63RD ST /62XX S VERNON AV,320R 312R 313R,NaN,400,E,63RD ST /62XX S VERNON,AV,None,NaN,NaN,EAST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171231,2215814431,06/07/2022 19.58.44,PERGUN,PERSON WITH A GUN,19PZ,23XX W 95TH ST / 94XX S WESTERN AV,4264A 2262A 2262D,NaN,2300,W,95TH ST / 94XX S WESTERN,AV,None,NaN,NaN,WEST
171234,2215814941,06/07/2022 20.28.16,DISTME,MENTAL HEALTH DISTURBANCE,19P,54XX N BROADWAY,2023 2011,E59 452 A61,5400,N,NaN,BROADWAY,None,NaN,NaN,NORTH
171245,2215816273,06/07/2022 21.38.45,SHOTSF,SHOTS FIRED,19PZ,58XX N BROADWAY / 11XX W ARDMORE AV,2020,NaN,5800,N,BROADWAY / 11XX W ARDMORE,AV,None,NaN,NaN,NORTH
171247,2215816336,06/07/2022 21.43.04,SUPPOR,SUPPORT UNIT REQUEST,5079Z,27XX N LAKE SHORE DR,1924 1934,NaN,2700,N,LAKE SHORE,DR,None,NaN,NaN,NORTH


In [95]:
for i in range(24,30):
    exec(f'df_results = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["directional"],x["clean_street_name"],database),axis=1)')
    df = df_results.to_frame(name='t')
    codes = df['t'].apply(extractElement1)
    df['zipcodes'] = codes
    
    lats = df['t'].apply(extractElement2)
    df['lats'] = lats
    
    longs = df['t'].apply(extractElement3)
    df['longs'] = longs
    
    df = df.drop('t', axis=1)
    

        #df_final = pd.DataFrame.from_records(df_results, columns=['zipcodes','lats','longs'])
    df_complete = pd.concat([chunkDf[i], df], axis=1)
        #df_complete['zipcode'] = df_complete['zipcodes'].apply(extractElement)
        #df_complete['lat'] = df_complete['lats'].apply(extractElement)
        #df_complete['long'] = df_complete['longs'].apply(extractElement)
    oemc_output = pd.concat([oemc_output,df_complete])

    print('Done')

Done
Done
Done
Done
Done
Done


In [113]:
oemc_output.to_csv('oemc_output_data2.csv')

In [114]:
oemc_output.shape

(171247, 16)

In [115]:
oemc_output

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,clean_street_name
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,SOUTH,HALSTED,ST,60620,41.755904,-87.643854,HALSTED
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,SOUTH,SPAULDING,AV,60655,41.704913,-87.704104,SPAULDING
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,WEST,SCHOOL,ST,60641,41.940710,-87.729486,SCHOOL
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,None,NaN,NaN,N SPAULDING
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,WEST,IRVING PARK,RD,60634,41.953793,-87.790068,IRVING PARK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171204,2215812221,06/07/2022 17.47.42,DISTME,MENTAL HEALTH DISTURBANCE,19E,40XX N WESTERN AV / 24XX W CUYLER AV,1921,NaN,4000,NORTH,WESTERN AV / 24XX W CUYLER,AV,60618,41.955359,-87.687555,WESTERN
171208,2215812568,06/07/2022 18.09.58,DISTME,MENTAL HEALTH DISTURBANCE,5B,23XX W DIVERSEY AV / 28XX N OAKLEY AV,1433 1421,NaN,2300,WEST,DIVERSEY AV / 28XX N OAKLEY,AV,60647,41.932394,-87.686286,DIVERSEY
171231,2215814431,06/07/2022 19.58.44,PERGUN,PERSON WITH A GUN,19PZ,23XX W 95TH ST / 94XX S WESTERN AV,4264A 2262A 2262D,NaN,2300,WEST,95TH ST / 94XX S WESTERN,AV,60643,41.720689,-87.680166,95TH
171245,2215816273,06/07/2022 21.38.45,SHOTSF,SHOTS FIRED,19PZ,58XX N BROADWAY / 11XX W ARDMORE AV,2020,NaN,5800,NORTH,BROADWAY / 11XX W ARDMORE,AV,60660,41.988814,-87.660624,BROADWAY


In [116]:
t = pd.read_csv('oemc_output_data2.csv')
t.shape

(171247, 17)

In [98]:
oemc_output[oemc_output['zipcodes'].isnull()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,clean_street_name
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,None,NaN,NaN,N SPAULDING
26,2000103846,01/01/2020 03.28.07,DISTME,MENTAL HEALTH DISTURBANCE,15P,91XX S DR MARTIN LUTHER KING JR DR,632R,NaN,9100,SOUTH,DR MARTIN LUTHER KING JR,DR,None,NaN,NaN,DR MARTIN LUTHER KING JR
40,2000105688,01/01/2020 06.04.23,EMS,EMS,19B,75XX N PAULINA ST,2410 2431 2423,NaN,7500,NORTH,PAULINA,ST,None,NaN,NaN,PAULINA
46,2000106684,01/01/2020 08.11.37,MISSPE,MISSING PERSON,6050,50XX S DR MARTIN LUTHER KING JR DR,220 223 222,NaN,5000,SOUTH,DR MARTIN LUTHER KING JR,DR,None,NaN,NaN,DR MARTIN LUTHER KING JR
71,2000108852,01/01/2020 11.52.52,CHECWB,CHECK WELL BEING,19P,3XX E 55TH,231 242,NaN,300,EAST,NaN,55TH,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170892,2215700338,06/06/2022 00.27.35,CHECWB,CHECK WELL BEING,19B,11XX E 75TH ST / 74XX S SOUTH CHICAGO AV,324,NaN,1100,EAST,75TH ST / 74XX S SOUTH CHICAGO,AV,None,NaN,NaN,75TH
171000,2215709739,06/06/2022 15.29.39,DISTME,MENTAL HEALTH DISTURBANCE,19B,23XX W 68TH ST / 67XX S WESTERN AV,832,NaN,2300,WEST,68TH ST / 67XX S WESTERN,AV,None,NaN,NaN,68TH
171007,2215710175,06/06/2022 15.51.54,DISTME,MENTAL HEALTH DISTURBANCE,9P,1XX E MONROE DR / X S MICHIGAN AV,130,T6 A74,100,EAST,MONROE DR / X S MICHIGAN,AV,None,NaN,NaN,MONROE
171047,2215713675,06/06/2022 19.31.28,DISTME,MENTAL HEALTH DISTURBANCE,19BZ,11XX E 80TH ST / 80XX S WOODLAWN AV,412,NaN,1100,EAST,80TH ST / 80XX S WOODLAWN,AV,None,NaN,NaN,80TH


In [79]:
28404/171247

0.16586567939876318

In [99]:
13738/171247

0.08022330318195356

In [100]:
100-0.08


99.92

In [70]:
import numpy as np


In [71]:
oemc_output.zipcodes.fillna(value=np.nan, inplace=True)


In [72]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,60620,41.755904,-87.643854,SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,60655,41.704913,-87.704104,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,60641,41.940710,-87.729486,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,NaN,NaN,NaN,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,60634,41.953793,-87.790068,WEST


In [73]:
oemc_output.dtypes

EventNumber          object
EntryDate            object
EventType            object
TypeDescription      object
FinalDisposition     object
Location             object
CPDUnitList          object
CFDUnitList          object
numerical            object
directional          object
street_name          object
suffix               object
zipcodes             object
lats                float64
longs               float64
St_PreDir            object
dtype: object

In [77]:
oemc_output['zipcodes'] = pd.to_numeric(oemc_output['zipcodes'])


In [78]:
oemc_output.dtypes

EventNumber          object
EntryDate            object
EventType            object
TypeDescription      object
FinalDisposition     object
Location             object
CPDUnitList          object
CFDUnitList          object
numerical            object
directional          object
street_name          object
suffix               object
zipcodes            float64
lats                float64
longs               float64
St_PreDir            object
dtype: object

In [79]:
oemc_output.to_csv('final_clean_geocodeds_batch.csv')

In [75]:
oemc_output.zipcodes = oemc_output.zipcodes.astype(int)

ValueError: cannot convert float NaN to integer

In [206]:
df

,zipcodes,lats,longs
344,60615,41.806610,41.806610
345,None,NaN,NaN
346,60610,41.907583,41.907583
347,60624,41.865941,41.865941
348,60629,41.772362,41.772362
...,...,...,...
511,60644,41.881185,41.881185
512,60610,41.908214,41.908214
513,60660,41.989067,41.989067
514,60618,41.953589,41.953589


In [209]:
df_complete

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes,lats,longs
344,2000215069,01/02/2020 21.48.42,CHECWB,CHECK WELL BEING,19B,48XX S COTTAGE GROVE AV,224,NaN,4800,S,COTTAGE GROVE,AV,SOUTH,60615,41.806610,41.806610
345,2000215080,01/02/2020 21.49.39,CHECWB,CHECK WELL BEING,19PZ,15XX W FULLERTON AV /24XX N ASHLAND AV,1932 1920,NaN,1500,W,FULLERTON AV /24XX N ASHLAND,AV,WEST,None,NaN,NaN
346,2000215083,01/02/2020 21.49.46,DISTME,MENTAL HEALTH DISTURBANCE,5IZ,13XX N LAKE SHORE DR,1806A 1831R,NaN,1300,N,LAKE SHORE,DR,NORTH,60610,41.907583,41.907583
347,2000215087,01/02/2020 21.50.12,DIST,DISTURBANCE,4PZ,40XX W ROOSEVELT RD,1011A,NaN,4000,W,ROOSEVELT,RD,WEST,60624,41.865941,41.865941
348,2000215283,01/02/2020 22.05.46,ATSUCD,ATTEMPT SUICIDE,15IZ,66XX S KEDVALE AV,841R 833R,NaN,6600,S,KEDVALE,AV,SOUTH,60629,41.772362,41.772362
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2000314787,01/03/2020 20.35.21,DISTME,MENTAL HEALTH DISTURBANCE,19P,X N LOCKWOOD AV,1523,NaN,0,N,LOCKWOOD,AV,NORTH,60644,41.881185,41.881185
512,2000314921,01/03/2020 20.43.23,CHECWB,CHECK WELL BEING,19B,14XX N NORTH PARK AV,1872,NaN,1400,N,NORTH PARK,AV,NORTH,60610,41.908214,41.908214
513,2000315061,01/03/2020 20.51.38,THREAS,THREATENING SUICIDE,19B,59XX N HERMITAGE AV,2012,NaN,5900,N,HERMITAGE,AV,NORTH,60660,41.989067,41.989067
514,2000315112,01/03/2020 20.55.09,DIST,DISTURBANCE,5FZ,33XX W IRVING PARK RD,1732,NaN,3300,W,IRVING PARK,RD,WEST,60618,41.953589,41.953589


In [211]:
oemc_output

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,St_PreDir,zipcode,lat,long
0,2.000100e+09,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600.0,S,HALSTED,ST,[60620],"[41.75590425, 41.75534803, 41.75439881, 41.755...","[-87.64385433, -87.64383799, -87.64450703, -87...",SOUTH,60620,41.755904,-87.643854
1,2.000100e+09,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300.0,S,SPAULDING,AV,[60655],"[41.70491317, 41.70578963, 41.70479904, 41.704...","[-87.70410419, -87.70413558, -87.70410952, -87...",SOUTH,60655,41.704913,-87.704104
2,2.000101e+09,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000.0,W,SCHOOL,ST,[60641],"[41.94070996, 41.94066717, 41.94066687, 41.940...","[-87.72948623, -87.72867574, -87.72931246, -87...",WEST,60641,41.940710,-87.729486
3,2.000101e+09,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100.0,.5,N SPAULDING,AV,[],[],[],.5,None,NaN,NaN
4,2.000101e+09,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500.0,W,IRVING PARK,RD,[60634],"[41.95379298, 41.95332232, 41.95332206, 41.953...","[-87.7900676, -87.79091283, -87.79039137, -87....",WEST,60634,41.953793,-87.790068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,2.000315e+09,01/03/2020 20.35.21,DISTME,MENTAL HEALTH DISTURBANCE,19P,X N LOCKWOOD AV,1523,NaN,0.0,N,LOCKWOOD,AV,60644,41.881185,41.881185,NORTH,NaN,NaN,NaN
512,2.000315e+09,01/03/2020 20.43.23,CHECWB,CHECK WELL BEING,19B,14XX N NORTH PARK AV,1872,NaN,1400.0,N,NORTH PARK,AV,60610,41.908214,41.908214,NORTH,NaN,NaN,NaN
513,2.000315e+09,01/03/2020 20.51.38,THREAS,THREATENING SUICIDE,19B,59XX N HERMITAGE AV,2012,NaN,5900.0,N,HERMITAGE,AV,60660,41.989067,41.989067,NORTH,NaN,NaN,NaN
514,2.000315e+09,01/03/2020 20.55.09,DIST,DISTURBANCE,5FZ,33XX W IRVING PARK RD,1732,NaN,3300.0,W,IRVING PARK,RD,60618,41.953589,41.953589,WEST,NaN,NaN,NaN


In [150]:
for i in range(6):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
    

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


...

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [151]:
oemc_output.shape

(39963, 14)

In [154]:
for i in range(6,12):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

Done


In [155]:
oemc_output.shape

(74212, 14)

In [ ]:
oemc_output.to_csv('oemc_geocoded_final_batch.csv')

In [157]:
for i in range(12,18):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

Done


In [158]:
oemc_output.shape

(108460, 14)

In [159]:
for i in range(18,24):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

Done


In [160]:
oemc_output.shape

(142708, 14)

In [161]:
for i in range(24,30):
    exec(f'list_zipcodes{i} = chunkDf[{i}].apply(lambda x: geoCodeChunk(x["numerical"],x["St_PreDir"],x["street_name"],database),axis=1)')
    exec(f'geoCodeToDf(i,chunkDf,list_zipcodes{i})')
    exec(f'oemc_output = pd.concat([oemc_output,chunkDf[{i}]])')
print('Done')

Done


In [162]:
oemc_output.shape

(176956, 14)

In [163]:
oemc_output.to_csv('oemc_geocoded_final_batch.csv')

In [164]:
oemc_output.shape

(176956, 14)

In [165]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,[60620],SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,[60655],SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,"[60641, 60618]",WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,[],.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,[60634],WEST


In [ ]:
#Extract the first zip codes from the zipcodes column

In [170]:
def extractZip(zipCode):
    try:
        return zipCode[0]
    except:
        return None
    

In [171]:
oemc_output['cleanZipCodes'] = oemc_output['zipcodes'].apply(extractZip)

In [172]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,St_PreDir,cleanZipCodes
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,[60620],SOUTH,60620
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,[60655],SOUTH,60655
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,"[60641, 60618]",WEST,60641
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,[],.5,None
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,[60634],WEST,60634


In [90]:
oemc_output[oemc_output['zipcodes'].isnull()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,St_PreDir
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,NaN,NaN,NaN,.5
16,2000102204,01/01/2020 01.59.07,THREAS,THREATENING SUICIDE,19P,5XX N CLARK ST /1XX W GRAND AV,1860,NaN,500,N,CLARK ST /1XX W GRAND,AV,NaN,NaN,NaN,NORTH
26,2000103846,01/01/2020 03.28.07,DISTME,MENTAL HEALTH DISTURBANCE,15P,91XX S DR MARTIN LUTHER KING JR DR,632R,NaN,9100,S,DR MARTIN LUTHER KING JR,DR,NaN,NaN,NaN,SOUTH
28,2000104170,01/01/2020 03.47.33,THREAS,THREATENING SUICIDE,19B,7XX E 87TH ST /87XX S COTTAGE GROVE AV,631R,NaN,700,E,87TH ST /87XX S COTTAGE GROVE,AV,NaN,NaN,NaN,EAST
37,2000105293,01/01/2020 05.19.12,BATIP,BATTERY IP,19IZ,4XX E 63RD ST /62XX S VERNON AV,320R 312R 313R,NaN,400,E,63RD ST /62XX S VERNON,AV,NaN,NaN,NaN,EAST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171231,2215814431,06/07/2022 19.58.44,PERGUN,PERSON WITH A GUN,19PZ,23XX W 95TH ST / 94XX S WESTERN AV,4264A 2262A 2262D,NaN,2300,W,95TH ST / 94XX S WESTERN,AV,NaN,NaN,NaN,WEST
171234,2215814941,06/07/2022 20.28.16,DISTME,MENTAL HEALTH DISTURBANCE,19P,54XX N BROADWAY,2023 2011,E59 452 A61,5400,N,NaN,BROADWAY,NaN,NaN,NaN,NORTH
171245,2215816273,06/07/2022 21.38.45,SHOTSF,SHOTS FIRED,19PZ,58XX N BROADWAY / 11XX W ARDMORE AV,2020,NaN,5800,N,BROADWAY / 11XX W ARDMORE,AV,NaN,NaN,NaN,NORTH
171247,2215816336,06/07/2022 21.43.04,SUPPOR,SUPPORT UNIT REQUEST,5079Z,27XX N LAKE SHORE DR,1924 1934,NaN,2700,N,LAKE SHORE,DR,NaN,NaN,NaN,NORTH


In [91]:
28498/176956

0.1610456836727774

In [176]:
oemc_output.to_csv('cleaned_oemc_geocoded_final_batch.csv')

In [80]:
oemc_output.shape

(171247, 17)

In [81]:
27409/176956

0.15489161147403874

In [82]:
oemc_output[oemc_output['zipcodes'].isnull()]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,St_PreDir,stripedZipCodes
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,None,NaN,NaN,.5,None
16,2000102204,01/01/2020 01.59.07,THREAS,THREATENING SUICIDE,19P,5XX N CLARK ST /1XX W GRAND AV,1860,NaN,500,N,CLARK ST /1XX W GRAND,AV,None,NaN,NaN,NORTH,None
26,2000103846,01/01/2020 03.28.07,DISTME,MENTAL HEALTH DISTURBANCE,15P,91XX S DR MARTIN LUTHER KING JR DR,632R,NaN,9100,S,DR MARTIN LUTHER KING JR,DR,None,NaN,NaN,SOUTH,None
28,2000104170,01/01/2020 03.47.33,THREAS,THREATENING SUICIDE,19B,7XX E 87TH ST /87XX S COTTAGE GROVE AV,631R,NaN,700,E,87TH ST /87XX S COTTAGE GROVE,AV,None,NaN,NaN,EAST,None
37,2000105293,01/01/2020 05.19.12,BATIP,BATTERY IP,19IZ,4XX E 63RD ST /62XX S VERNON AV,320R 312R 313R,NaN,400,E,63RD ST /62XX S VERNON,AV,None,NaN,NaN,EAST,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171231,2215814431,06/07/2022 19.58.44,PERGUN,PERSON WITH A GUN,19PZ,23XX W 95TH ST / 94XX S WESTERN AV,4264A 2262A 2262D,NaN,2300,W,95TH ST / 94XX S WESTERN,AV,None,NaN,NaN,WEST,None
171234,2215814941,06/07/2022 20.28.16,DISTME,MENTAL HEALTH DISTURBANCE,19P,54XX N BROADWAY,2023 2011,E59 452 A61,5400,N,NaN,BROADWAY,None,NaN,NaN,NORTH,None
171245,2215816273,06/07/2022 21.38.45,SHOTSF,SHOTS FIRED,19PZ,58XX N BROADWAY / 11XX W ARDMORE AV,2020,NaN,5800,N,BROADWAY / 11XX W ARDMORE,AV,None,NaN,NaN,NORTH,None
171247,2215816336,06/07/2022 21.43.04,SUPPOR,SUPPORT UNIT REQUEST,5079Z,27XX N LAKE SHORE DR,1924 1934,NaN,2700,N,LAKE SHORE,DR,None,NaN,NaN,NORTH,None


In [71]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,60620,41.755904,41.755904,SOUTH
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,60655,41.704913,41.704913,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,60641,41.940710,41.940710,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,None,NaN,NaN,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,60634,41.953793,41.953793,WEST


In [72]:
#turn zipcodes into a string 
oemc_output['stripedZipCodes'] = oemc_output['zipcodes'].apply(str)

In [80]:
oemc_output['zipcodes'].value_counts()

60619.0    8494
60620.0    6663
60649.0    6248
60617.0    6154
60628.0    6016
60626.0    4699
60644.0    4610
60640.0    4245
60629.0    4197
60636.0    4079
60651.0    3947
60624.0    3929
60623.0    3509
60621.0    3471
60637.0    3434
60609.0    3208
60608.0    3038
60639.0    2784
60643.0    2783
60647.0    2770
60612.0    2767
60625.0    2664
60618.0    2655
60653.0    2622
60641.0    2398
60615.0    2366
60610.0    2340
60657.0    2292
60660.0    2269
60632.0    2177
60616.0    1948
60614.0    1919
60607.0    1913
60611.0    1893
60634.0    1837
60622.0    1805
60613.0    1707
60659.0    1664
60605.0    1607
60645.0    1534
60630.0    1467
60638.0    1336
60652.0    1319
60601.0    1086
60654.0     806
60642.0     678
60656.0     564
60707.0     555
60827.0     531
60655.0     507
60631.0     505
60661.0     480
60602.0     475
60646.0     472
60604.0     366
60633.0     348
60606.0     316
60603.0     283
Name: zipcodes, dtype: int64

In [81]:
oemc_output[oemc_output['TypeDescription'] == 'ATTEMPT SUICIDE']['zipcodes'].value_counts()[0:50]

KeyError: 0

In [200]:
oemc_output.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,St_PreDir,cleanZipCodes,stripedZipCodes
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,[60620],SOUTH,60620,60620
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,[60655],SOUTH,60655,60655
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,"[60641, 60618]",WEST,60641,60641
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,[],.5,None,None
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,[60634],WEST,60634,60634


In [177]:
oemc_output['cleanZipCodes'].value_counts()

60619    8870
60649    6578
60617    6208
60620    5501
60626    5318
         ... 
60660      36
60612      31
60603      18
60643       5
60616       1
Name: cleanZipCodes, Length: 93, dtype: int64

In [178]:
oemc_output['TypeDescription'].value_counts()[0:20]

MENTAL HEALTH DISTURBANCE    59118
CHECK WELL BEING             22226
THREATENING SUICIDE          15469
DOMESTIC DISTURBANCE         12528
DISTURBANCE                  10839
EMS                           6416
ASSAULT IP                    4046
PERSON WITH A KNIFE           3187
ATTEMPT SUICIDE               2892
DOMESTIC BATTERY              2831
DISTURBANCE - MUSIC/NOISE     2475
CRIMINAL TRESPASS IP          2379
SUSPICIOUS PERSON             2343
BATTERY IP                    2272
MISSING PERSON                2093
NO POLICE SERVICE             2091
PERSON WITH A GUN             1924
SEX OFFENSE OTHER             1212
ASSAULT JO                    1088
BATTERY JO                    1056
Name: TypeDescription, dtype: int64

In [82]:
mental_disturbance = oemc_output[oemc_output['TypeDescription'] == 'MENTAL HEALTH DISTURBANCE']
mental_disturbance.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,zipcodes,lats,longs,St_PreDir
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,60620.0,41.755904,-87.643854,SOUTH
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,NaN,NaN,NaN,.5
8,2000100999,01/01/2020 00.52.12,DISTME,MENTAL HEALTH DISTURBANCE,5079Z,50XX S MICHIGAN AV,224R,NaN,5000,S,MICHIGAN,AV,60615.0,41.802480,-87.622291,SOUTH
12,2000101523,01/01/2020 01.20.34,DISTME,MENTAL HEALTH DISTURBANCE,7I,51XX N MILWAUKEE AV,1623R,NaN,5100,N,MILWAUKEE,AV,60630.0,41.973731,-87.765940,NORTH
15,2000101661,01/01/2020 01.28.59,DISTME,MENTAL HEALTH DISTURBANCE,19P,77XX S SAWYER AV,851R 852R,NaN,7700,S,SAWYER,AV,60652.0,41.753027,-87.703975,SOUTH


In [83]:
mental_disturbance['zipcodes'].value_counts()

60620.0    2292
60628.0    2174
60649.0    2032
60617.0    2027
60619.0    1967
60644.0    1831
60636.0    1580
60651.0    1547
60624.0    1507
60640.0    1468
60626.0    1443
60629.0    1275
60637.0    1270
60623.0    1243
60621.0    1224
60609.0    1075
60612.0    1068
60639.0    1020
60643.0    1002
60608.0     994
60647.0     956
60618.0     911
60615.0     887
60610.0     839
60625.0     838
60641.0     834
60653.0     834
60607.0     703
60657.0     671
60632.0     653
60660.0     630
60634.0     624
60614.0     609
60611.0     606
60622.0     605
60616.0     581
60659.0     552
60645.0     543
60613.0     523
60605.0     518
60652.0     512
60630.0     461
60638.0     449
60601.0     330
60642.0     251
60654.0     249
60707.0     228
60827.0     179
60655.0     170
60661.0     157
60646.0     154
60602.0     151
60631.0     147
60656.0     138
60604.0     115
60603.0     100
60606.0      95
60633.0      89
Name: zipcodes, dtype: int64

In [84]:
threatening = oemc_output[oemc_output['TypeDescription'] == 'THREATENING SUICIDE']
threatening['zipcodes'].value_counts()

60640.0    452
60620.0    431
60619.0    422
60649.0    399
60626.0    396
60628.0    387
60623.0    363
60608.0    345
60629.0    344
60624.0    340
60617.0    337
60609.0    335
60641.0    324
60644.0    313
60639.0    311
60618.0    309
60647.0    296
60651.0    283
60636.0    279
60611.0    277
60607.0    275
60637.0    273
60657.0    270
60632.0    256
60621.0    248
60622.0    245
60612.0    243
60625.0    233
60610.0    230
60616.0    222
60613.0    220
60660.0    215
60634.0    194
60614.0    183
60615.0    176
60630.0    175
60605.0    170
60653.0    165
60643.0    162
60659.0    146
60638.0    141
60645.0    136
60652.0    112
60601.0    105
60654.0     85
60642.0     85
60656.0     68
60631.0     68
60661.0     65
60633.0     50
60655.0     47
60646.0     46
60827.0     43
60707.0     43
60604.0     39
60602.0     35
60606.0     26
60603.0     21
Name: zipcodes, dtype: int64

In [85]:
domestic_disturbance = oemc_output[oemc_output['TypeDescription'] == 'DOMESTIC DISTURBANCE']
domestic_disturbance['zipcodes'].value_counts()

60620.0    808
60619.0    800
60617.0    738
60649.0    701
60628.0    670
60629.0    476
60651.0    473
60644.0    436
60636.0    412
60624.0    380
60637.0    357
60623.0    336
60643.0    317
60647.0    309
60621.0    303
60609.0    299
60653.0    273
60639.0    226
60626.0    209
60632.0    195
60608.0    188
60612.0    177
60641.0    172
60618.0    158
60634.0    157
60615.0    153
60625.0    129
60652.0    126
60638.0    123
60659.0    122
60640.0    120
60630.0    107
60645.0    106
60616.0     87
60827.0     76
60622.0     71
60660.0     69
60657.0     54
60613.0     51
60610.0     50
60707.0     49
60655.0     47
60656.0     44
60614.0     41
60631.0     31
60633.0     25
60607.0     24
60642.0     21
60611.0     20
60646.0     20
60605.0     19
60601.0     12
60654.0      9
60661.0      7
60606.0      4
60602.0      2
60603.0      2
60604.0      1
Name: zipcodes, dtype: int64

In [86]:
disturbance = oemc_output[oemc_output['TypeDescription'] == 'DISTURBANCE']
disturbance['zipcodes'].value_counts()

60619.0    451
60649.0    396
60617.0    370
60626.0    365
60620.0    322
60640.0    321
60628.0    321
60657.0    236
60636.0    230
60629.0    226
60608.0    215
60653.0    215
60621.0    208
60611.0    202
60614.0    201
60615.0    197
60610.0    193
60605.0    186
60643.0    184
60647.0    178
60601.0    177
60625.0    172
60618.0    171
60609.0    163
60612.0    162
60607.0    161
60644.0    154
60637.0    149
60624.0    148
60651.0    147
60616.0    146
60613.0    141
60659.0    128
60654.0    126
60660.0    119
60630.0    117
60639.0    116
60632.0    108
60622.0    104
60623.0     99
60641.0     91
60602.0     83
60604.0     78
60645.0     62
60642.0     62
60634.0     60
60603.0     57
60661.0     51
60606.0     51
60638.0     47
60652.0     44
60656.0     38
60646.0     22
60631.0     22
60655.0     18
60633.0     16
60827.0     15
60707.0     13
Name: zipcodes, dtype: int64

In [87]:
attempt_suicide = oemc_output[oemc_output['TypeDescription'] == 'ATTEMPT SUICIDE']
attempt_suicide['zipcodes'].value_counts()

60619.0    105
60644.0     90
60620.0     88
60629.0     86
60623.0     84
60647.0     77
60624.0     76
60649.0     73
60618.0     69
60609.0     67
60641.0     67
60632.0     66
60625.0     65
60637.0     64
60628.0     64
60617.0     64
60640.0     62
60639.0     61
60626.0     60
60651.0     56
60621.0     56
60636.0     55
60643.0     52
60608.0     52
60634.0     46
60613.0     45
60622.0     43
60638.0     42
60660.0     42
60611.0     40
60614.0     40
60610.0     39
60630.0     37
60607.0     37
60657.0     37
60659.0     33
60616.0     33
60615.0     33
60645.0     32
60612.0     31
60653.0     29
60652.0     22
60605.0     21
60642.0     17
60646.0     16
60656.0     15
60655.0     14
60601.0     14
60707.0     14
60633.0     12
60827.0     12
60654.0     11
60631.0     10
60604.0      5
60661.0      5
60602.0      4
60606.0      3
60603.0      1
Name: zipcodes, dtype: int64

In [88]:
attempt_suicide['zipcodes'].value_counts()

60619.0    105
60644.0     90
60620.0     88
60629.0     86
60623.0     84
60647.0     77
60624.0     76
60649.0     73
60618.0     69
60609.0     67
60641.0     67
60632.0     66
60625.0     65
60637.0     64
60628.0     64
60617.0     64
60640.0     62
60639.0     61
60626.0     60
60651.0     56
60621.0     56
60636.0     55
60643.0     52
60608.0     52
60634.0     46
60613.0     45
60622.0     43
60638.0     42
60660.0     42
60611.0     40
60614.0     40
60610.0     39
60630.0     37
60607.0     37
60657.0     37
60659.0     33
60616.0     33
60615.0     33
60645.0     32
60612.0     31
60653.0     29
60652.0     22
60605.0     21
60642.0     17
60646.0     16
60656.0     15
60655.0     14
60601.0     14
60707.0     14
60633.0     12
60827.0     12
60654.0     11
60631.0     10
60604.0      5
60661.0      5
60602.0      4
60606.0      3
60603.0      1
Name: zipcodes, dtype: int64

In [89]:
oemc_output[oemc_output['TypeDescription'] == 'ATTEMPT SUICIDE']['zipcodes'].value_counts()[0:50]

KeyError: 0

## Testing code

In [141]:
list_zipcodes0

0              [60620]
1              [60655]
2       [60641, 60618]
3                   []
4              [60634]
             ...      
5705           [60615]
5706    [60629, 60629]
5707    [60610, 60642]
5708           [60626]
5709           [60644]
Length: 5709, dtype: object

In [146]:
geoCodeToDf(0,chunkDf,list_zipcodes0)

In [147]:
chunkDf[0]

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes
0,2000100015,01/01/2020 00.00.52,DISTME,MENTAL HEALTH DISTURBANCE,19B,76XX S HALSTED ST,621R,NaN,7600,S,HALSTED,ST,SOUTH,[60620]
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655]
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,"[60641, 60618]"
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[]
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5705,2003312092,02/02/2020 17.59.38,DISTME,MENTAL HEALTH DISTURBANCE,15I,11XX E 50TH ST,222,NaN,1100,E,50TH,ST,EAST,[60615]
5706,2003312108,02/02/2020 18.00.43,ASLTIP,ASSAULT IP,5B,59XX S SAWYER AV,823,NaN,5900,S,SAWYER,AV,SOUTH,"[60629, 60629]"
5707,2003312147,02/02/2020 18.03.29,DIST,DISTURBANCE,5PZ,15XX N CLYBOURN AV,1822,NaN,1500,N,CLYBOURN,AV,NORTH,"[60610, 60642]"
5708,2003312242,02/02/2020 18.11.31,CHECWB,CHECK WELL BEING,19PZ,14XX W PRATT BL,2433 2432,NaN,1400,W,PRATT,BL,WEST,[60626]


In [124]:
testChunk

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655]
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,"[60641, 60618]"
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[]
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634]
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH,[60640]
6,2000100941,01/01/2020 00.49.19,BATIP,BATTERY IP,1EZ,66XX W BELDEN AV,2512R 2510R,NaN,6600,W,BELDEN,AV,WEST,[60707]
7,2000100959,01/01/2020 00.50.02,THREAS,THREATENING SUICIDE,16I,5XX N AUSTIN BL,1511R 1512R,NaN,500,N,AUSTIN,BL,NORTH,[60644]
8,2000100999,01/01/2020 00.52.12,DISTME,MENTAL HEALTH DISTURBANCE,5079Z,50XX S MICHIGAN AV,224R,NaN,5000,S,MICHIGAN,AV,SOUTH,[60615]
9,2000101032,01/01/2020 00.53.27,DD,DOMESTIC DISTURBANCE,1IZ,10XX N DEARBORN ST,1824R 1823R,NaN,1000,N,DEARBORN,ST,NORTH,[60610]


In [123]:
test_zipcodes

1           [60655]
2    [60641, 60618]
3                []
4           [60634]
5           [60640]
6           [60707]
7           [60644]
8           [60615]
9           [60610]
dtype: object

In [117]:
test_zipcodes = testChunk.apply(lambda x: geoCodeChunk(x['numerical'],x['St_PreDir'],x['street_name'],database),axis=1)

Done
--- 0.0023156404495239258 minutes ---
Done
--- 0.0017972787221272786 minutes ---
Done
--- 0.0016391714413960775 minutes ---
Done
--- 0.0017383178075154623 minutes ---
Done
--- 0.0019673307736714682 minutes ---
Done
--- 0.0017877737681070963 minutes ---
Done
--- 0.0018693049748738607 minutes ---
Done
--- 0.0019369761149088541 minutes ---
Done
--- 0.002000558376312256 minutes ---


In [118]:
testChunk['zipcodes'] = test_zipcodes

/tmp/ipykernel_451751/4203125367.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testChunk['zipcodes'] = test_zipcodes


In [93]:
testChunk.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical            int64
directional         object
street_name         object
suffix              object
St_PreDir           object
dtype: object

In [96]:
test_zipcodes = testChunk.apply(lambda x: geoCodeChunk(x['numerical'],x['St_PreDir'],x['street_name'],database),axis=1)

Done
--- 0.0021781166394551593 minutes ---
Done
--- 0.002099589506785075 minutes ---
Done
--- 0.0016209999720255533 minutes ---
Done
--- 0.0017386754353841147 minutes ---
Done
--- 0.001985267798105876 minutes ---
Done
--- 0.0017796754837036133 minutes ---
Done
--- 0.001896850268046061 minutes ---
Done
--- 0.0019377748171488444 minutes ---
Done
--- 0.0018540263175964356 minutes ---


In [90]:
test_zipcodes

1           [60655]
2    [60641, 60618]
3                []
4           [60634]
5           [60640]
6           [60707]
7           [60644]
8           [60615]
9           [60610]
dtype: object

In [81]:
testChunk = chunkDf[0].iloc[1:10]
testChunk.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH


In [47]:
#future use case this will not work for exact address
#do not search for address bewtween 100 off
def geoCodeChunk(number,chunkDfName,database):
    list_zipcodes = []
    for index, row in chunkDfName[number].iterrows():
        complete_conditional = database[(database['St_Name'] == row['street_name']) 
                                    & (database['St_PreDir'] == row['St_PreDir'])
                                    & (database['Add_Number'] < row.numerical + 100)
                                    & (database['Add_Number'] >= row.numerical -100)
                                   ]
        list_zipcodes.append(list(set(complete_conditional['Post_Code'])))
    #print(set(complete_conditional['Post_Code']))
    print('Done')
    print("--- %s minutes ---" % ((time.time() - start_time)/60))
    return list_zipcodes
    

In [48]:
def geoCodeToCSV(number,chunkDfName,chunkZipCodeList):
    chunkDfName[number]['zipcodes']= chunkZipCodeList
    chunkDfName[number].to_csv(f'geocoded_batches/batch{number}.csv')
    

In [49]:
#creates 30 empty list 
#name convention list_zipcodes0, list_zipcodes1,..etc
for k in range(30):
    exec(f'list_zipcodes{k} = []')


In [54]:
testChunk = chunkDf[0].iloc[1:10]
testChunk.head()

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH


In [ ]:
geoCodeChunk(number,chunkDfName,database)

In [71]:
test_zipcodes = []
for index, row in testChunk.iterrows():
    complete_conditional = database[(database['St_Name'] == row['street_name']) 
                                & (database['St_PreDir'] == row['St_PreDir'])
                                & (database['Add_Number'] < row.numerical + 100)
                                & (database['Add_Number'] >= row.numerical -100)
                                ]
    test_zipcodes.append(list(set(complete_conditional['Post_Code'])))
    #print(set(complete_conditional['Post_Code']))
print('Done')
#print("--- %s minutes ---" % ((time.time() - start_time)/60))


Done


In [ ]:
from ast import literal_eval
df = pd.read_csv('test.csv', converters={'col1': literal_eval})


In [57]:
testChunk['zipcodes'] = test_zipcodes
testChunk.to_csv(f'geocoded_batches/batch_test.csv')


/tmp/ipykernel_441706/893343377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testChunk['zipcodes'] = test_zipcodes


In [59]:
testChunk.dtypes

EventNumber          int64
EntryDate           object
EventType           object
TypeDescription     object
FinalDisposition    object
Location            object
CPDUnitList         object
CFDUnitList         object
numerical            int64
directional         object
street_name         object
suffix              object
St_PreDir           object
zipcodes            object
dtype: object

In [60]:
testChunk

,EventNumber,EntryDate,EventType,TypeDescription,FinalDisposition,Location,CPDUnitList,CFDUnitList,numerical,directional,street_name,suffix,St_PreDir,zipcodes
1,2000100451,01/01/2020 00.21.18,DIST,DISTURBANCE,4PZ,103XX S SPAULDING AV,2211R 2213R,NaN,10300,S,SPAULDING,AV,SOUTH,[60655]
2,2000100656,01/01/2020 00.33.55,DOMBAT,DOMESTIC BATTERY,5079Z,40XX W SCHOOL ST,X1720 1772R 1722R 1731R 1762B 1733R 1761 1762,NaN,4000,W,SCHOOL,ST,WEST,"[60641, 60618]"
3,2000100660,01/01/2020 00.34.00,DISTME,MENTAL HEALTH DISTURBANCE,19PZ,31XX .5 N SPAULDING AV,1413R,NaN,3100,.5,N SPAULDING,AV,.5,[]
4,2000100902,01/01/2020 00.47.02,CHECWB,CHECK WELL BEING,550,65XX W IRVING PARK RD,1631R,NaN,6500,W,IRVING PARK,RD,WEST,[60634]
5,2000100939,01/01/2020 00.49.17,CHECWB,CHECK WELL BEING,19B,50XX N SHERIDAN RD,2041R 2024R,NaN,5000,N,SHERIDAN,RD,NORTH,[60640]
6,2000100941,01/01/2020 00.49.19,BATIP,BATTERY IP,1EZ,66XX W BELDEN AV,2512R 2510R,NaN,6600,W,BELDEN,AV,WEST,[60707]
7,2000100959,01/01/2020 00.50.02,THREAS,THREATENING SUICIDE,16I,5XX N AUSTIN BL,1511R 1512R,NaN,500,N,AUSTIN,BL,NORTH,[60644]
8,2000100999,01/01/2020 00.52.12,DISTME,MENTAL HEALTH DISTURBANCE,5079Z,50XX S MICHIGAN AV,224R,NaN,5000,S,MICHIGAN,AV,SOUTH,[60615]
9,2000101032,01/01/2020 00.53.27,DD,DOMESTIC DISTURBANCE,1IZ,10XX N DEARBORN ST,1824R 1823R,NaN,1000,N,DEARBORN,ST,NORTH,[60610]


In [69]:
testChunk[['zipcodes']].iloc[0][0][0]

60655

In [70]:
testChunk[['zipcodes']].iloc[1][0][0]

60641